In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from datetime import datetime

In [5]:
options = Options()
options.set_capability('se:name', 'Screp Reclamacoes')

In [6]:
driver = webdriver.Remote(options=options,command_executor="http://localhost:4444")

In [7]:
driver.get("https://consumidor.gov.br/pages/indicador/geral/abrir")

In [8]:
wait = WebDriverWait(driver, 60)

In [9]:
wait.until(EC.element_to_be_clickable((By.ID, 'li_tab_relatos')))

<selenium.webdriver.remote.webelement.WebElement (session="ea734d1a-b8ce-417a-8ff5-bcb0ef7af977", element="97abd1fc-430a-403a-9b2e-a7c319876b75")>

In [10]:
tab_relatos = driver.find_element(By.ID, "li_tab_relatos")

In [11]:
tab_relatos.click()

In [12]:
wait.until(EC.presence_of_element_located((By.ID, 'contador')))

<selenium.webdriver.remote.webelement.WebElement (session="ea734d1a-b8ce-417a-8ff5-bcb0ef7af977", element="89311e3b-e20d-4d38-8439-45b5609f2943")>

In [13]:
contador = driver.find_element(By.ID, 'contador')

In [14]:
data_hora = contador.text.split('desde')[1].strip().replace(',', '')

In [15]:
data_atu = datetime.strptime(data_hora, '%d/%m/%Y %H:%M:%S')

In [16]:
try:
    with open('../csv/data_hora.bst', encoding='utf8') as file:
        data_pas = file.readline()
        data_pas = datetime.strptime(data_pas, '%d/%m/%Y %H:%M:%S')
except:
    with open('../csv/data_hora.bst', "w") as file:
        file.write(data_hora)
        data_pas = data_atu

In [17]:
if data_atu<=data_pas:
    print("Não há atualização")
    driver.close()
    #exit(0)

In [18]:
while data_atu>data_pas:
    wait.until(EC.element_to_be_clickable((By.ID, 'btn-mais-resultados')))
    btn_mais_resultados = driver.find_element(By.ID, "btn-mais-resultados")
    btn_mais_resultados.click()
    
    contador = driver.find_element(By.ID, 'contador')
    data_hora_l = contador.text.split('desde')[1].strip().replace(',', '')    
    data_atu = datetime.strptime(data_hora_l, '%d/%m/%Y %H:%M:%S')    

In [19]:
with open('../csv/data_hora.bst', "w") as file:
    file.write(data_hora)

In [ ]:
resultados = driver.find_element(By.ID, 'resultados')

In [21]:
nom_empresas = resultados.find_elements(By.CLASS_NAME, 'relatos-nome-empresa')


In [22]:
empresas = []

In [23]:
for empresa in nom_empresas:
    empresas.append(empresa.find_element(By.TAG_NAME, 'a').text)    

In [24]:
rel_status = resultados.find_elements(By.TAG_NAME, 'h4')

In [25]:
lst_status = []

In [26]:
for status in rel_status:
    lst_status.append(status.text)        

In [27]:
rel_datas = resultados.find_elements(By.CLASS_NAME, 'relatos-data')

In [28]:
tempo_respostas = []
datas = []
cidades = []
ufs = []
count = 1

In [29]:
for data in rel_datas:
    if count % 2 == 0:
        tempo_respostas.append(data.text.replace('(','').replace(')',''))   
    else:
        datas.append(data.text.split(',')[0])
        cidades.append(data.text.split(',')[1].split('-')[0].strip())
        ufs.append(data.text.split(',')[1].split('-')[1].strip())
    count=count+1

In [30]:
lst_relatos = resultados.find_elements(By.XPATH, "//div[3]/p")
lst_respostas = resultados.find_elements(By.XPATH, "//div[4]/p")
lst_notas_comentarios = resultados.find_elements(By.XPATH, "//div[5]/p")

In [31]:
relatos = []
respostas = []
notas = []
comentarios = []

In [32]:
for relato in lst_relatos:
    relatos.append(relato.text)        

In [33]:
for resposta in lst_respostas:
    respostas.append(resposta.text)     

In [34]:
count = 1
for nota in lst_notas_comentarios:
    if count % 2 == 0:
        comentarios.append(nota.text)
    else:
        notas.append(nota.text)
    count=count+1

In [35]:
relatos = list(zip(empresas,lst_status,tempo_respostas, datas, cidades, ufs, relatos, respostas, notas, comentarios))

In [41]:
relatos

[('Cemig',
  'Resolvido',
  'no mesmo dia',
  '14/10/2024',
  'Contagem',
  'MG',
  'Qual o prazo para análise de carga?',
  'Olá, Felipe! Boa tarde, tudo bem? Em atenção à sua manifestação, informo que o prazo de análise de carga pode variar de 20 a 30 dias para ser finalizada, é importante frisa que o senhor pode está solicitando a análise de carga através dos nossos canais digitais disponibilizados. A Cemig agradece a sua confiança, desejamos a você e toda sua família uma ótima tarde. Reiteramos que permanecemos a disposição para esclarecimentos de quaisquer duvidas Atenciosamente E.C Equipe de experiência Cemig.',
  'Nota 5',
  '<não há comentários do consumidor>'),
 ('Cemig',
  'Resolvido',
  'no mesmo dia',
  '14/10/2024',
  'Belo Horizonte',
  'MG',
  'não concordo com a taxa de iluminação publica',
  'Olá, Ana! Boa Tarde, tudo bem? Em resposta a sua manifestação no site consumidor.gov, VOCÊ SABIA QUE A CIP – CONTRIBUIÇÃO DE ILUMINAÇÃO PÚBLICA É DEFINIDA POR SEU MUNICÍPIO? De ac

In [37]:
driver.close()

In [42]:
from kafka import KafkaProducer

In [43]:
producer = KafkaProducer(bootstrap_servers=["localhost:9092"])


In [44]:
producer.send("reclamacoes", str(relatos).encode())